In [1]:
import os
if os.getcwd().endswith('lab04_bayes'):
    os.chdir('..')

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from lab04_bayes.utils import data

# Для начала посмотрим на данные в какой-то из частей

In [4]:
part1 = data.read_from_dir('lab04_bayes/resources/part1')
print(part1[0])
print(part1[1])

[(['23340', '1967', '12372', '1967', '130', '23172', '9887'], ['19054', '7634', '1847', '2130', '6049', '130', '9887', '2130', '20010', '118', '20048', '1804', '118', '6782', '19604', '118', '1593', '13025', '6335', '16922', '23265', '82', '18844', '23917', '16147', '17345', '14338', '130', '9887', '2130', '23172', '84', '23266', '24097', '80', '16502', '20010', '10092', '9561']), (['2056', '118', '10911'], ['4533', '4407', '1967', '14875', '118', '1781', '1847', '6485', '1835', '92', '22299', '2170', '123', '15346', '14535', '1613', '1640', '80', '8159', '1781', '709', '12982', '1967', '16502', '18851', '11852', '68', '21798', '21216', '180', '10670', '11666', '24731', '17173', '4651', '8234', '80', '13761', '70', '84', '196', '14373', '18877', '2785', '22299', '14535', '1613', '2531', '80', '22299', '13006', '13406', '9079', '84', '6485', '1835', '15221', '8696', '13406', '9110', '82', '14544', '24327', '7875', '5265', '180', '15372', '9472', '6240', '86', '18862', '84', '9676', '82'

Считывание работает, можно перейти к обработке:

# Классификатор

In [5]:
parts = data.read_all()

In [6]:
def train_test_split(test_part):
    train_parts = parts[:test_part] + parts[test_part + 1:]
    train = [[], []]
    for part in train_parts:
        train[0] += part[0]
        train[1] += part[1]
    return train, parts[test_part]

Построим для примера какой-нибудь классификатор:

In [7]:
def map_ngrams(d, n):
    result = []
    for cls in range(2):
        result.append(list(map(lambda t: data.collect_ngrams(t[0], t[1], n), d[cls])))
    return result

In [8]:
map_ngrams([parts[0][0][:10], parts[0][1][:10]], 4)

[[{'118+1593+13025+6335',
   '118+20048+1804+118',
   '118+6782+19604+118',
   '12372-1967-130-23172',
   '130+9887+2130+20010',
   '130+9887+2130+23172',
   '13025+6335+16922+23265',
   '14338+130+9887+2130',
   '1593+13025+6335+16922',
   '16147+17345+14338+130',
   '16922+23265+82+18844',
   '17345+14338+130+9887',
   '1804+118+6782+19604',
   '1847+2130+6049+130',
   '18844+23917+16147+17345',
   '19054+7634+1847+2130',
   '19604+118+1593+13025',
   '1967-12372-1967-130',
   '20010+118+20048+1804',
   '20048+1804+118+6782',
   '2130+20010+118+20048',
   '2130+23172+84+23266',
   '2130+6049+130+9887',
   '23172+84+23266+24097',
   '23265+82+18844+23917',
   '23266+24097+80+16502',
   '23340-1967-12372-1967',
   '23917+16147+17345+14338',
   '24097+80+16502+20010',
   '6049+130+9887+2130',
   '6335+16922+23265+82',
   '6782+19604+118+1593',
   '7634+1847+2130+6049',
   '80+16502+20010+10092',
   '82+18844+23917+16147',
   '84+23266+24097+80',
   '9887+2130+20010+118',
   '9887+2130+2

In [9]:
from lab04_bayes.bayes import BayesianClassifier

In [10]:
cls = BayesianClassifier(2, 0.5, [3, 1])
train, test = train_test_split(9)
cls.fit(map_ngrams(train, 3))

In [12]:
test_ngrams = map_ngrams(test, 3)
pred = [
    cls.predict(test_ngrams[0], verbose=True),
    cls.predict(test_ngrams[1], verbose=True)
]
acc = (np.sum(pred[0][:, 0] > pred[0][:, 1]) + np.sum(pred[1][:, 1] > pred[1][:, 0])) / (len(test_ngrams[0]) + len(test_ngrams[1]))
print(acc)

Total values: 61
- 1 processed
- 10 processed
- 20 processed
- 30 processed
- 40 processed
- 50 processed
- 60 processed
Total values: 48
- 1 processed
- 10 processed
- 20 processed
- 30 processed
- 40 processed
0.6330275229357798


In [14]:
ok0 = np.sum(pred[0][:, 0] > pred[0][:, 1])
ok1 = np.sum(pred[1][:, 1] > pred[1][:, 0])

In [15]:
ok0, len(test_ngrams[0]), ok1, len(test_ngrams[1])

(56, 61, 13, 48)

Как видно, смещение лямбды в сторону "хороших" сообщений позволило нам достаточно часто угадывать их, но при этом пропускать много спама. Теперь подберем параметры так, чтобы добиться точности 100% на не-спам письмах.

# Подбор параметров

Кажется логичным, что общая суммарная accuracy зависит от alpha и n сильнее, чем от lambda (а именно - если найдем модель с наилучшими alpha и n, и для нее будем двигать lambda, то скорее всего получим лучший результат)

In [22]:
def ngram_parts(n):
    return list(map(lambda p: map_ngrams(p, n), parts))

def train_test_split(parts2, test_part):
    train_parts = parts2[:test_part] + parts2[test_part + 1:]
    train = [[], []]
    for part in train_parts:
        train[0] += part[0]
        train[1] += part[1]
    return train, parts2[test_part]

In [23]:
for n in [1, 2, 3]:
    parts2 = ngram_parts(n)
    for alpha in [0.01, 0.1, 0.5]:
        print(f'With a={alpha}, n={n}:')
        scores = []
        for i in range(10):
            train, test = train_test_split(parts2, i)
            cls = BayesianClassifier(2, alpha, [1, 1])
            cls.fit(train)
            pred = [
                cls.predict(test[0]),
                cls.predict(test[1])
            ]
            ok0 = np.sum(pred[0][:, 0] > pred[0][:, 1])
            ok1 = np.sum(pred[1][:, 1] > pred[1][:, 0])
            scores.append((ok0 / len(test[0]), ok1 / len(test[1]), (ok0 + ok1) / (len(test[0]) + len(test[1]))))
            print(f'- group {i + 1} done')
        sc0 = np.mean([scores[i][0] for i in range(10)])
        sc1 = np.mean([scores[i][1] for i in range(10)])
        sct = np.mean([scores[i][2] for i in range(10)])
        print(f'- {sc0}/{sc1} - {sct}')

With a=0.01, n=1:
- group 1 done
- group 2 done
- group 3 done
- group 4 done
- group 5 done
- group 6 done
- group 7 done
- group 8 done
- group 9 done
- group 10 done
- 0.9704918032786886/0.8791666666666668 - 0.9302752293577982
With a=0.1, n=1:
- group 1 done
- group 2 done
- group 3 done
- group 4 done
- group 5 done
- group 6 done
- group 7 done
- group 8 done
- group 9 done
- group 10 done
- 0.980327868852459/0.9041666666666666 - 0.9467889908256881
With a=0.5, n=1:
- group 1 done
- group 2 done
- group 3 done
- group 4 done
- group 5 done
- group 6 done
- group 7 done
- group 8 done
- group 9 done
- group 10 done
- 0.9901639344262294/0.8708333333333333 - 0.9376146788990827
With a=0.01, n=2:
- group 1 done
- group 2 done
- group 3 done
- group 4 done
- group 5 done
- group 6 done
- group 7 done
- group 8 done
- group 9 done
- group 10 done
- 0.7786885245901639/0.6291666666666667 - 0.7128440366972477
With a=0.1, n=2:
- group 1 done
- group 2 done
- group 3 done
- group 4 done
- grou

KeyboardInterrupt: 

In [33]:
n = 1
alpha = 0.5

## Подборка штрафа

In [ ]:
parts1 = ngram_parts(n)
print(f'lambdas [1, 1]:')
for i in range(10):
    train, test = train_test_split(parts1, i)
    cls = BayesianClassifier(2, alpha, [1, 1])
    cls.fit(train)
    pred = [
        cls.predict(test[0], compute_probs=False),
        cls.predict(test[1], compute_probs=False)
    ]
    ok0 = np.sum(pred[0][:, 0] > pred[0][:, 1])
    if ok0 < len(test[0]):
        print(f'- p{i + 1}: {ok0}, {len(test[0])}: {np.max(pred[0][:, 1] - pred[0][:, 0])}')

lambdas [1, 1]:


In [ ]:
from math import exp

def quality(log_lambda):
    scores = []
    for i in range(10):
        train, test = train_test_split(parts1, i)
        cls = BayesianClassifier(2, alpha, [exp(81), 1])
        cls.fit(train)
        pred = [
            cls.predict(test[0], compute_probs=False),
            cls.predict(test[1], compute_probs=False)
        ]
        ok0 = np.sum(pred[0][:, 0] > pred[0][:, 1])
        ok1 = np.sum(pred[1][:, 1] > pred[1][:, 0])
        scores.append((ok0 / len(test[0]), ok1 / len(test[1]), (ok0 + ok1) / (len(test[0]) + len(test[1]))))
        print(f'- group {i + 1} done')
    sc0 = np.mean([scores[i][0] for i in range(10)])
    sc1 = np.mean([scores[i][1] for i in range(10)])
    sct = np.mean([scores[i][2] for i in range(10)])
    return sc0, sc1, sct

quality(81)